# **Desarrollar un Servicio Web de Traducción con Flask y Hugging Face**

Emily Chimbo

15/03/2025

El objetivo de este ejercicio es crear un servicio web utilizando Flask que se conecte a un modelo de traducción de Hugging Face y lo exponga a través de una API RESTful. Debe desarrollar un servicio que permita traducir texto entre dos idiomas utilizando un modelo preentrenado de Hugging Face.

Requisitos:
1. Tecnologías a utilizar:
 • Flask: Para crear un servicio web en Python.
 • Hugging Face Transformers: Para utilizar un modelo preentrenado de traducción.
 • Python 3.x: Para el desarrollo del servicio web.
2. Modelo de Hugging Face:
 • Deberán elegir un modelo de traducción disponible en Hugging Face (por ejemplo, Helsinki-NLP/opus-mt-en-es para traducir de inglés a español).
3. Requisitos del servicio web:
 • Endpoint principal: Un endpoint /translate que acepte solicitudes POST con un texto en formato JSON y devuelva el texto traducido.
 • Funcionamiento básico: El servicio debe traducir el texto de un idioma a otro usando el modelo de Hugging Face.
4. Requisitos opcionales (extra: hasta un punto adicional):
 • Despliegue en la nube: Para los  interesados, pueden pedir que desplieguen el servicio en una plataforma de la nube como Heroku, AWS, o Google Cloud. Esto les permitirá poner en práctica conceptos de despliegue de servicios web en un entorno de producción.

Entregables:
 1. Código fuente de la aplicación en github.
 2. Instrucciones de uso que expliquen cómo ejecutar el servicio localmente o en la nube.

In [1]:
!pip install flask transformers pyngrok

In [2]:
from flask import Flask, request, jsonify
from transformers import pipeline, AutoModelForSeq2SeqLM, AutoTokenizer
from pyngrok import ngrok, conf
import os

In [3]:
app = Flask(__name__)

ModelosIdioma = {
    ("en", "es"): "Helsinki-NLP/opus-mt-en-es",
    ("es", "en"): "Helsinki-NLP/opus-mt-es-en"
}

ModeloQuechua = "somosnlp-hackathon-2022/t5-small-finetuned-spanish-to-quechua"
ModeloQuechuaT5 = AutoModelForSeq2SeqLM.from_pretrained(ModeloQuechua)
TokenizerQuechuaT5 = AutoTokenizer.from_pretrained(ModeloQuechua)

def traducir_a_quechua(texto):
    input_ids = TokenizerQuechuaT5(texto, return_tensors="pt").input_ids
    outputs = ModeloQuechuaT5.generate(input_ids, max_length=40, num_beams=4, early_stopping=True)
    texto_traducido = TokenizerQuechuaT5.decode(outputs[0], skip_special_tokens=True)
    return texto_traducido

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

In [ ]:
@app.route('/traducir', methods=['POST'])
def traducir_texto():
    try:
        data = request.get_json()
        texto = data.get('texto')
        idioma_origen = data.get('origen', 'es')
        idioma_destino = data.get('destino', 'en')

        if not texto:
            return jsonify({'error': 'No se proporcionó texto para traducir'}), 400

        # Si la traducción es es a qu (no hay de qu a es)
        if idioma_origen == "es" and idioma_destino == "qu":
            texto_traducido = traducir_a_quechua(texto)
            return jsonify({'texto traducido': texto_traducido})

        # Si el idioma está de es a en y viceversa
        modelo_hf = ModelosIdioma.get((idioma_origen, idioma_destino))
        if modelo_hf:
            traductor = pipeline("translation", model=modelo_hf)
            texto_traducido = traductor(texto)[0]['translation_text']
            return jsonify({'texto traducido': texto_traducido})

        return jsonify({'error': f'La traducción de {idioma_origen} a {idioma_destino} no está disponible'}), 400

    except Exception as e:
        return jsonify({'error': str(e)}), 500

# Configurar y exponer con ngrok
auth_token = "2uBx2HJrVP43UBblns7CdIGxIQC_63y1fC4LCWEAbS5w9i6mG"
ngrok.set_auth_token(auth_token)
public_url = ngrok.connect(5000).public_url
print(f"Tu API pública está disponible en: {public_url}/traducir")

if __name__ == '__main__':
    app.run(port=5000)

Tu API pública está disponible en: https://2989-35-245-209-8.ngrok-free.app/traducir
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


config.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/826k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.59M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu
INFO:werkzeug:127.0.0.1 - - [16/Mar/2025 00:38:43] "POST /traducir HTTP/1.1" 200 -
